In [69]:
# %load_ext autoreload
# %autoreload 2
%load_ext tuna

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import numpy as np
import dill
import os
import seaborn as sns
import torch
import pandas as pd

from collections import namedtuple
from hmmlearn.hmm import MultinomialHMM
from scipy.special import logsumexp
from scipy.stats import multivariate_normal
from spn.algorithms.Inference import log_likelihood

# from rspnlib.models import RSPN, EMTrainingParams
# from rspnlib.models.hmm import HmmRSPN
# from rspnlib.models.iohmm import (
#     IoHmmRSPN,
#     ExtendedIoHmmRSPN,
#     ContinuousIoHmmRSPN,
#     OverparametrisedIoHmmRSPN,
#     OverparametrisedContinuousIoHmmRSPN,
#     ExtendedOverparametrisedContinuousIoHmmRSPN,
# )
# from rspnlib.evaluation import (
#     compute_metrics,
#     get_empty_metrics,
#     add_metrics,
#     print_aggregate_metrics,
#     evaluate_rspn_one_step_ahead_prediction,
#     evaluate_rspn_observation_regression,
# )
# from experiment_utils.datasets.vasopressors_dataset import load_vasopressors_dataset
from experiment_utils.datasets.mimic_continuous_dataset import load_mimic_dataset
# from experiment_utils.baselines.lstm import train_eval_lstm
# from experiment_utils.baselines.most_common_class import train_eval_most_common_baseline


from engine_dataset import generate_engine_dataset, generate_engine_datasets

from oSLRAU_run import get_data
from spn.structure.leaves.parametric.Parametric import Gaussian, Bernoulli, Categorical,  In_Latent, Multivariate_Gaussian
from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.io.Graphics import plot_spn
import numpy as np
from spn.algorithms.Inference import log_likelihood
from sklearn.model_selection import train_test_split
from spn.algorithms.oSLRAU import oSLRAU, oSLRAUParams
from spn.algorithms.RSPN import RSPN
from spn.algorithms.TransformStructure import Prune ,Prune_oSLRAU

from hmmlearn import hmm
from spn.algorithms.Inference import log_likelihood

The tuna extension is already loaded. To reload it, use:
  %reload_ext tuna


# MIMIC Experiments

In [70]:
SEQ_LENGTH = 8
NUM_STATES = 20
NUM_SEEDS = 5
SEEDS = range(42, 42 + NUM_SEEDS)

# train_mimic_datasets, test_mimic_datasets = load_mimic_dataset(
#     seq_length=SEQ_LENGTH,
# )
# num_inputs = (
#     int(
#         np.max(train_mimic_datasets[0].inputs_raw)
#     )
#     + 1
# )
# observations_dim = 3

In [71]:
import pickle
train_mimic_datasets, test_mimic_datasets = pickle.load(open('./data_splits/data.pkl', 'rb'))
observations_dim = 3
num_inputs = 2

<h2> oSLRAU  </h2>

In [79]:
def train_rspn_mimic(train_data, test_data, epoch, rspn=None, os_args=None):

    if len(train_data.shape) == 3: train_data = np.squeeze(train_data)
    if len(test_data.shape) == 3: test_data = np.squeeze(test_data)

    if rspn is None:
    
        rspn = RSPN(num_variables=os_args["num_variables"], num_latent_variables=os_args["num_latent_variables"], num_latent_values=os_args["num_latent_values"])
        first_mini_batch = train_data[0:os_args["mini_batch_size"]]
        n = first_mini_batch.shape[1]

        print(f"n: {n}") # = num_time_steps_mix * n_dim
        
        context = [Gaussian, Gaussian, Gaussian, Categorical]*(int(n/4)) # Bernoulli Categorical
        ds_context = Context(parametric_types=context).add_domains(first_mini_batch[:, :os_args["num_variables"]])
        spn, initial_template_spn, top_spn = rspn.build_initial_template(first_mini_batch, ds_context, os_args["len_sequence_varies"])

    
    no_of_minibatches = int(train_data.shape[0] / os_args["mini_batch_size"])

    print(f"no of minibatches: {no_of_minibatches}")

    
    for i in range(1, no_of_minibatches):
        mini_batch = train_data[i * os_args["mini_batch_size"]: (i+1) * os_args["mini_batch_size"]]
    
        update_template = False
        
        # if (i % os_args["update_after_no_min_batches"] == 0) and (i!=0) :
        #if i % os_args["update_after_no_min_batches"] == 0 and i<=60 and epoch == 0:
        if i == 10 and epoch == 0:
            print(i)
            update_template = True

        
        template_spn = rspn.learn_rspn(mini_batch, update_template, os_args["os_params"], os_args["unroll"],
                                            os_args["full_update"], os_args["update_leaves"],
                                            os_args["len_sequence_varies"])
        
        # except Exception:
        #     print(f"==>> mini_batch: {mini_batch}")
        #     assert 0 == 1
           
            
    test_ll = np.sum(rspn.log_likelihood(test_data, os_args["unroll"], os_args["len_sequence_varies"]))


    return test_ll, rspn, template_spn

In [80]:
def final_ll_mimic(data, num_epochs=1, do_plot_spn=True, os_args=None, split_data=False):
    ll = -np.inf
    rspn=None
    if split_data:
        train_data, val_data= train_test_split(data, test_size=0.1, random_state=42)
    
    else:
        train_data, val_data = data, data
    for epoch in range(num_epochs):
        print(train_data.shape)
        
        
        epoch_ll, rspn, template_spn = train_rspn_mimic(train_data, val_data, epoch, rspn, os_args)        
        print(f"epoch_ll: {epoch_ll}")
        
        ll = epoch_ll
        del template_spn

    print(f" \n\n\n final ll: {ll}\n\n\n")

    if do_plot_spn:
        plot_spn(template_spn, 'rspn_final_template.pdf')
    
    return rspn

In [81]:
# np.unique(train_mimic_datasets[0].inputs_raw)

#train_mimic_datasets[0].observations_raw_continuous.shape

In [82]:
def process_ds(ds):
    raw_inputs = ds.inputs_raw
    obs_raw_cont = ds.observations_raw_continuous

    comb_ds = []
    for i in range(raw_inputs.shape[0]):
        comb_ds_i = []
        for j in range(raw_inputs.shape[1]):
             comb_ds_j = [*list(obs_raw_cont[i, j*3: (j+1)*3]), raw_inputs[i,j]]
             comb_ds_i.append(comb_ds_j)

        comb_ds.append(comb_ds_i)

    comb_ds = np.array(comb_ds)
    comb_ds = comb_ds.reshape(raw_inputs.shape[0], -1)

    return comb_ds


In [83]:
n_states = 2
n_dim = 4

num_time_steps = 8

oSLRAU_params = oSLRAUParams(mergebatch_threshold=128, corrthresh=0.1, mvmaxscope=1, equalweight=True, currVals=True)

os_args = {
    "num_variables": num_time_steps * n_dim,
    "num_latent_variables" : 4,
    "num_latent_values" : 4,
    "unroll" : "backward",
    "full_update" : False,
    "update_leaves" : True,
    "len_sequence_varies" : False,
    "mini_batch_size" : 5,
    "update_after_no_min_batches" : 20,
    "os_params": oSLRAU_params
    }

In [84]:
log_likelihoods = []
rspn_mimic_models = []
train_ds_list = []
test_ds_list = []

In [86]:
from experiment_utils.datasets.mimic_continuous_dataset import load_mimic_dataset
train_mimic_datasets, test_mimic_datasets = load_mimic_dataset(seq_length=8)
print(train_mimic_datasets[0].inputs_raw[-2])

  0%|          | 0/6052 [00:00<?, ?it/s]

Loaded 5967 data points
Partitioning into folds...
Done!
[0 0 3 2 2 2 2 3]


In [85]:
base_seed = SEEDS[0]
seed_dict = {SEEDS[i]: i for i in range(len(SEEDS))}
seed_ls = SEEDS # [SEEDS[1]]
#del rspn_mimic

for seed in seed_ls:
    i = seed_dict[seed]
    print(f"——————————[ Running experiment for seed {seed} ]——————————")
    train_ds, test_ds = process_ds(train_mimic_datasets[i]), process_ds(test_mimic_datasets[i])
    train_ds_list.append(train_ds)
    test_ds_list.append(test_ds)
    rspn_mimic = final_ll_mimic(train_ds, num_epochs=3, do_plot_spn=False, os_args=os_args)
    rspn_test_ll_mimic = np.sum(rspn_mimic.log_likelihood(test_ds, os_args["unroll"], os_args["len_sequence_varies"]))
    print(f"seed {seed}: log likelihood: {rspn_test_ll_mimic}")
    log_likelihoods.append(rspn_test_ll_mimic)
    rspn_mimic_models.append(rspn_mimic)

——————————[ Running experiment for seed 42 ]——————————
(4773, 32)
n: 32
==>> data.shape: (5, 32)
len(self.meta_types): 32
self.meta_types.values(): dict_values([<MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.REAL: 1>, <MetaType.DISCRETE: 3>])
len(self.meta_types.values()): 32
data[:, col]: [115.   65.   82.   86.5 110. ]
==>> domain_values: [65.0, 115.0]
feature_meta_type: MetaType.REAL
==>> domain_values:

/Users/dariyankhan/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Work (one drive)/Imperial/MIT/exchange_year/summer-UROP-knowledge-distillation/policy_stuff/policy-learn/SPFlow/src/spn/algorithms/oSLRAU.py:76: RuntimeWarning: divide by zero encountered in log
  w_children_log_probs[:, i] = lls_per_node[parent_result, c.id] + np.log(node.weights[i])


==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 3, 4])}
==>> in_latent_dict: {In_LatentNode_1: array([0, 1, 2, 

KeyboardInterrupt: 

In [19]:
log_likelihoods

[-115144.99615650639,
 -111339.66783088766,
 -109088.83414411306,
 -108887.05396010102,
 -113035.58515876927]

In [20]:
# print(np.mean(log_likelihoods))
#log_likelihoods
print(f"Mean: {np.array(log_likelihoods).mean()}")
print(f"STD: {np.array(log_likelihoods).std()}")

Mean: -111499.22745007547
STD: 2379.534827359451


In [21]:
rspn_mimic_models

In [22]:
# def rmse(x, real_data):
#     diff_sq = (x- real_data)**2
#     return np.sqrt(np.mean(diff_sq, axis=1))

# a = np.array([[1,2,3], [4,5,6]])
# b = np.array([[1,2,3], [1,2,3]])
# rmse(a,b)



array([0., 3.])

In [42]:
# train_0 = train_mimic_datasets[0].observations_raw_continuous
# test_0 = train_0 + 1

# train_0_var_0 = train_0[:, 0::3].copy()
# train_0_var_1 = train_0[:, 1::3].copy()
# train_0_var_2 = train_0[:,2::3].copy()

# sk_mse(train_0.flatten(), test_0.flatten(), multioutput="raw_values", squared=False)




array([1.])

In [ ]:
# import pickle

# with open('all_mimic_vars.pickle', 'wb') as file:
#     pickle.dump(rspn_mimic_models, file)  # Store variable 'a'
#     pickle.dump(test_ds_list, file) 

#Initialize variables

In [ ]:
import pickle
with open('all_mimic_vars.pickle', 'rb') as file:  
    rspn_mimic_models = pickle.load(file)  
    test_ds_list = pickle.load(file)

In [64]:
from spn.algorithms.Marginalization import marginalize
from copy import deepcopy
from spn.algorithms.MPE import mpe
from spn.structure.Base import assign_ids, rebuild_scopes_bottom_up
from spn.io.Text import str_to_spn, spn_to_str_equation
from sklearn.metrics import mean_squared_error as sk_mse

num_time_steps = 8
n_dim = 4


def next_error(model, data, num_time_steps=num_time_steps, num_dims=n_dim):
    unrolled_model = model.get_unrolled_rspn(model.get_len_sequence())
    preds_arr = np.zeros((data.shape[0], (num_time_steps)*(n_dim-1))) # assuming 1 control variable

    preds_arr[:, :3] = data[:, :3]

    p = 3 #index to insert for preds array
    for k, t in enumerate(range(8, num_time_steps*n_dim+1, 4)):
        # print(f"__________{t}___________________")

        
        prev_inputs = list(range(3, t-4, 4))

        inputs_to_include = []
        for i in range(t):
            if i not in prev_inputs:
                inputs_to_include.append(i)

        spn_marg = marginalize(unrolled_model, inputs_to_include)
        
        data_t = deepcopy(data)
        data_t[:, [t-2, t-3, t-4]] = np.nan
        data_t = data_t[:, :t]

       
        spn_mpe = mpe(spn_marg, data_t)
        curr_spn_mpe = spn_mpe[:, [t-2, t-3, t-4]]

        print(curr_spn_mpe[:3])
        #assert False
        #curr_data = data[:, [t-2, t-3, t-4]]


        preds_arr[:, [p, p+1, p+2]] = curr_spn_mpe
        p += 3


    return preds_arr



def rmse_over_vars(preds, real_data):
    rmses = []
    for i in range(n_dim-1):
        data_sl = real_data[:, i::4]
        preds_sl = preds[:, i::3]


        rmse_i = sk_mse(preds_sl.flatten(), data_sl.flatten(), multioutput="raw_values", squared=True)
        rmses.append(rmse_i)
    
    return rmses



def all_rmses(models, test_ds_list):
    for i, (model, test_ds) in enumerate(zip(models, test_ds_list)):
        print(f"seed:__________{SEEDS[i]}___________")
        pred_arr = next_error(model, test_ds)
        rmse_vars = rmse_over_vars(pred_arr, test_ds)
        print(rmse_vars)
        print(np.mean(rmse_vars))



all_rmses(rspn_mimic_models, test_ds_list)    





seed:__________42___________
==>> child: In_LatentNode_1
==>> child: In_LatentNode_2
==>> child: In_LatentNode_139
==>> child: In_LatentNode_140
==>> child: In_LatentNode_141
==>> child: In_LatentNode_142
==>> child: In_LatentNode_143
==>> child: In_LatentNode_144
==>> child: In_LatentNode_145
==>> child: In_LatentNode_146
[[19.57894976 74.01918747 88.41899244]
 [19.57894976 74.01918747 88.41899244]
 [19.57894976 74.01918747 88.41899244]]
[[19.61245953 73.5844906  87.97281735]
 [19.61245953 73.5844906  87.97281735]
 [19.61245953 73.5844906  87.97281735]]
[[19.55260602 73.73627019 87.81527787]
 [19.55260602 73.73627019 87.81527787]
 [19.55260602 73.73627019 87.81527787]]
[[19.52185671 73.80378295 87.63021807]
 [19.52185671 73.80378295 87.63021807]
 [19.52185671 73.80378295 87.63021807]]
[[19.49257998 73.95884204 87.44850675]
 [19.49257998 73.95884204 87.44850675]
 [19.49257998 73.95884204 87.44850675]]
[[19.47590924 73.82836865 87.30242818]
 [19.47590924 73.82836865 87.30242818]
 [19.47

In [77]:
# rspn_mimic_models0 = rspn_mimic_models[0]
# test_ds0 = test_ds_list[0]

In [25]:
# import pickle

# with open('all_mimic_vars.pickle', 'wb') as file:
#     pickle.dump(rspn_mimic_models, file)  # Store variable 'a'
#     pickle.dump(test_ds_list, file) 

#Initialize variables




In [28]:
# %%capture
# all_rmse_arr = all_rmse_arrs(rspn_mimic_models, test_ds_list)




In [36]:
# mean_rmses = np.zeros((5, num_time_steps-1))
# for i, arr in enumerate(all_rmse_arr):
#     mean_rmse = np.mean(arr, axis=0)
#     mean_rmses[i,:] = mean_rmse

# print(mean_rmses)


[[14.32973613 14.51410039 14.08211036 13.92345384 14.03174623 13.76924325
  13.91715382]
 [12.10984279 11.91042356 11.74018355 11.5342395  11.64309678 11.64912231
  11.66630483]
 [11.28441641 10.76388689 10.67852185 10.82338683 10.6825062  10.613792
  10.76430437]
 [11.54198374 10.91345255 10.94969766 10.55146276 10.8300366  10.67696935
  10.47065633]
 [12.06567854 11.93587873 11.55630368 11.37074703 11.38719948 11.20166968
  11.06509124]]


In [22]:
np.unique(test_ds0[:,15], return_counts=True)

(array([0., 1.]), array([1144,   50]))